## function utilities

In [1]:
!pip install -q streamlit streamlit-folium sodapy openmeteo_requests requests_cache retry_requests plotly streamlit_js_eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.0/327.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import openmeteo_requests
import requests_cache
from retry_requests import retry

def fetch_om_data(latitude, longitude, freq):
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    url = "https://archive-api.open-meteo.com/v1/archive"

    value_param =  ["temperature_2m", "relative_humidity_2m", "rain", "direct_radiation"]

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": value_param,
        "start_date": "2020-06-16",
        "end_date": "2024-06-30",
        "timezone": "America/Chicago"
    }

    try:
        responses = openmeteo.weather_api(url, params=params)

        response = responses[0]

        data = response.Hourly()


        data_temperature_2m = data.Variables(0).ValuesAsNumpy()
        data_relative_humidity_2m = data.Variables(1).ValuesAsNumpy()
        data_rain = data.Variables(2).ValuesAsNumpy()
        data_direct_radiation = data.Variables(3).ValuesAsNumpy()

        data = {"date": pd.date_range(
            start = pd.to_datetime(data.Time(), unit = "s", utc = True),
            end = pd.to_datetime(data.TimeEnd(), unit = "s", utc = True),
            freq = pd.Timedelta(seconds = data.Interval()),
            inclusive = "left"
        )}

        data["temperature_2m"] = data_temperature_2m
        data["relative_humidity_2m"] = data_relative_humidity_2m
        data["rain"] = data_rain
        data["direct_radiation"] = data_direct_radiation

        dataframe = pd.DataFrame(data = data)

        if freq == "daily":
            daily_dataframe = dataframe.resample('D', on='date').mean().reset_index()
            return daily_dataframe
        elif freq == "monthly":
            daily_dataframe = dataframe.resample('D', on='date').mean().reset_index()
            monthly_dataframe = daily_dataframe.resample('M', on='date').mean().reset_index()
            return monthly_dataframe
        else:
            return dataframe

    except Exception as e:
        print(f"Failed to fetch data from OpenMeteo: {e}")
        return e




## app

In [17]:
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets
from IPython.display import display, clear_output
import IPython

import plotly.io as pio
pio.renderers.default = "colab"

def fetch_and_display_weather_data(lat, lng, freq, variable):
    weather_data = fetch_om_data(lat, lng, freq)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=weather_data['date'], y=weather_data[variable], mode='lines', name=variable.capitalize()))
    fig.update_layout(
        title=f'{variable.capitalize()} over Time ({freq.capitalize()}) at ({lat}, {lng})',
        xaxis_title='Time',
        yaxis_title=variable.capitalize()
    )
    fig.show(renderer="colab")

display(widgets.HTML("<h1 style='text-align: center; color: #fae6e6;'>Weather project</h1>"))

lat_input = widgets.FloatText(description='Latitude:')
lng_input = widgets.FloatText(description='Longitude:')

source_data = widgets.RadioButtons(options=['OpenMeteo', 'Dane', 'Local (upload file)'], description='Data Source:', disabled=False)
variable_selector = widgets.RadioButtons(options=["temperature_2m", "relative_humidity_2m", "rain", "direct_radiation"], description='Variable:', disabled=False)
freq_selector = widgets.RadioButtons(options=['hourly', 'daily', 'monthly'], description='Frequency:', disabled=False)

fetch_button = widgets.Button(description='Fetch Data')

output = widgets.Output()

def on_fetch_button_clicked(b):
      clear_output()
      lat = lat_input.value
      lng = lng_input.value
      if source_data.value == "OpenMeteo":
          fetch_and_display_weather_data(lat, lng, freq_selector.value, variable_selector.value)

fetch_button.on_click(on_fetch_button_clicked)

display(lat_input)
display(lng_input)
display(source_data)
display(variable_selector)
display(freq_selector)
display(fetch_button)
display(output)
